## Домашнее задание к Уроку 4

### Задание:

1. Развернуть у себя на компьютере/виртуальной машине/хостинге MongoDB и реализовать функцию, которая будет добавлять только новые вакансии/продукты в вашу базу.
2. Написать функцию, которая производит поиск и выводит на экран вакансии с заработной платой больше введённой суммы (необходимо анализировать оба поля зарплаты).

### Решение:

#### Импортируем необходимые библиотеки

In [26]:
import requests
from fake_useragent import UserAgent
from copy import deepcopy
from time import sleep
from pprint import pprint
import json
from pymongo import MongoClient
from pymongo.errors import DuplicateKeyError

#### Определяем необходимые переменные

In [2]:
# Зададим переменную поиска
KEY_SEARCH_STRING = 'Python'

# Зададим типовую структуру вакансии для наполнения списка вакансий
tmp_vacancy_dict = {
    # Наименование вакансии
    'vacancy': '',
    # Зарплата включая минимум, максимум и валюту
    'salary': {'min_salary': 0., 'max_salary': 0., 'currency': ''},
    # Ссылка на описание вакансии
    'vacancy_link': '',
    # источник вакансии
    'vacancy_source': 'hh.ru',
    # Наименование работодателя
    'name_of_employer': '',
    # Место работы (город или удаленка)
    'city_of_work': '',
    # условия работы - полный день, удаленка
    'type_of_work': ''
}

#### Определим необходимые функции

In [3]:
def get_headhunter_vacancies(headhunter_url, headhunter_request_params, npages):
    # С данного сайта получить данные вакансий можно через api в json формате
    # Инициализируем список словарей для хранения вакансий
    headhunter_vacancy_list = []

    # Организуем цикл сбора информации
    for i in range(npages):
        # Чтобы избежать ошибки о превышении количества запросов
        sleep(2)

        # Заносим в параметры запроса номер страницы
        headhunter_request_params['page'] = str(i)

        # Выполним запрос к сайту, ответ преобразуем в json а затем в список
        headhunter_respond = requests.get(url=headhunter_url, params=headhunter_request_params)
        headhunter_respond_json_list = (headhunter_respond.json())['items']

        if headhunter_respond.ok:
            # Для каждой вакансии
            for one_vacancy in headhunter_respond_json_list:
                # Заполняем словарь вакансии
                tmp_vacancy_dict['vacancy'] = one_vacancy['name']

                if one_vacancy['salary'] is None:
                    tmp_vacancy_dict['salary']['min_salary'] = 0.0
                    tmp_vacancy_dict['salary']['max_salary'] = 0.0
                    tmp_vacancy_dict['salary']['currency'] = ''
                else:
                    if one_vacancy['salary']['from'] is None:
                        tmp_vacancy_dict['salary']['min_salary'] = 0.0
                    else:
                        tmp_vacancy_dict['salary']['min_salary'] = float(one_vacancy['salary']['from'])

                    if one_vacancy['salary']['to'] is None:
                        tmp_vacancy_dict['salary']['max_salary'] = 0.0
                    else:
                        tmp_vacancy_dict['salary']['max_salary'] = float(one_vacancy['salary']['to'])

                    tmp_vacancy_dict['salary']['currency'] = one_vacancy['salary']['currency']

                tmp_vacancy_dict['vacancy_link'] = one_vacancy['alternate_url']
                tmp_vacancy_dict['name_of_employer'] = one_vacancy['employer']['name']
                tmp_vacancy_dict['city_of_work'] = one_vacancy['area']['name']
                tmp_vacancy_dict['type_of_work'] = one_vacancy['schedule']['name']

                # Добавляем вакансю в список вакансий
                headhunter_vacancy_list.append(deepcopy(tmp_vacancy_dict))
        else:
            break

    return headhunter_vacancy_list

In [4]:
def find_new_vacancy_and_insert_to_db(new_vacancy_list, vacancy_collection):
    
    # Выгрузим из БД имеющиеся вакансии без поля _id
    headhunter_db_vacancy_list = list(vacancy_collection.find({'vacancy': {'$exists': True}}, {'_id': False}))

    # Создадим пустой список вакансий для вставки в БД
    headhunter_diff_list_to_insertion = []

    # Для каждого словаря из полученного после парсинга списка
    for item in new_vacancy_list:
        # Проверим есть ли вакансия в БД
        if item not in headhunter_db_vacancy_list:
            # Если нет, добавим словарь в список для вставки
            headhunter_diff_list_to_insertion.append(item)

    # Если были обнаружены новые ваканчии то
    if len(headhunter_diff_list_to_insertion) > 0:
        # Внесем полученный список вакансий в БД
        x = headhunter_collection.insert_many(headhunter_diff_list_to_insertion)
        vacancy_count = len(headhunter_diff_list_to_insertion)
        return vacancy_count
    else:
        return 0

#### Основное тело программы

In [5]:
# Задаем url для запроса
headhunter_url = 'https://api.hh.ru/vacancies'

# Зададим словарь параметров для запросов на сайте HeadHunter
headhunter_request_params = {
    'text': KEY_SEARCH_STRING,
    'area': '1',
    'items_on_page': '20',
    'page': '0'
}

#### Подключимся и создадим структуру БД в MangoDB

In [6]:
#Подключимся к локальной MongoDB
local_mdb_client = MongoClient('mongodb://localhost:27017/')
# Подключимся к БД vacancy_db
vacancy_db = local_mdb_client['vacancy_db']
# Подключим коллекцию headhunter
headhunter_collection = vacancy_db['headhunter']

#### Обработка новых вакансий разработчика python для города Москвы с сайта HeadHunter

In [7]:
%%time
# Инициализируем список вакансий
headhunter_vacancy_list = []
# Инициализируем маркер ошибки
hh_err = False

# Получим список вакансий hh
try:
    headhunter_vacancy_list = get_headhunter_vacancies(headhunter_url, headhunter_request_params, npages=40)
except Exception as e:
    # печатаем ошибку
    err_msg = f'При обработке вакансий на headhunter.ru произошла ошибка: {e}'
    print(err_msg)
    hh_err = True

if not hh_err:
    # Запишем в базу MangoDB vacancy_db словари с вакансиями hh
    # Что бы избежать дубликатов вакансий запись в базу, проверим что в БД нет таких вакансий
    
    new_vacancy_update_number = find_new_vacancy_and_insert_to_db(headhunter_vacancy_list, headhunter_collection)
    
    if new_vacancy_update_number > 0:
        print(f'В БД vacancy_db было добавлено {new_vacancy_update_number} новых вакансий!')
        print(f'Всего вакансий в БД vacancy_db.headhunter {len(list(headhunter_collection.find({})))}')
    else:
        print(f'На сайте headhunter.ru не появилось новых вакансий!')

    # Очищаем все списки данных
    #headhunter_vacancy_list.clear()
    #headhunter_diff_list_to_insertion.clear()


В БД vacancy_db было добавлено 20 новых вакансий!
Всего вакансий в БД vacancy_db.headhunter 820
Wall time: 1min 36s


In [40]:
# Проверим количество вакансий в БД на данный момент
len(list(headhunter_collection.find()))

820

#### Поиск и вывод на экран вакансии с заработной платой больше введённой суммы

In [43]:
# Зададим функция поиска и вывода вакансий из БД по параметру величины зарплаты
# По умолчанию будут выводиться зарплаты больше указанного размера зарплаты (volume_of_salary).
# При изменнении is_gt на False будут найдены вакансии меньше указаного обьема зарплаты
def print_vacancy_by_salary_number(vacancy_collection, volume_of_salary, is_gt = True):
    
    # Определим кретерий поиска: больше или меньше указаного обьма зарплаты
    if is_gt:
        operator = '$gt'
        operator_symbol = '>'
    else:
        operator = '$lt'
        operator_symbol = '<'
    
    # Зададим стороку поиска
    vacancy_query_dict = {
        "salary.min_salary": {operator: volume_of_salary}, 
        "salary.max_salary": {operator: volume_of_salary}
    }
    
    # Выполним поиск вакансий по заданным кретериям
    vacancy_with_requested_salary_cursor = vacancy_collection.find(vacancy_query_dict)
    
    finded_vacancy_count = len(list(vacancy_collection.find(vacancy_query_dict)))
    
    print(f'Количество найденных вакансий с размером зарплаты {operator_symbol} {volume_of_salary}: {finded_vacancy_count}')
    print('Список найденных вакансий:\n')
    for item in vacancy_with_requested_salary_cursor:
        pprint(item)
    

In [44]:
print_vacancy_by_salary_number(headhunter_collection, 100000)

Количество найденных вакансий с размером зарплаты > 100000: 55
Список найденных вакансий:

{'_id': ObjectId('63f8d278121c6a068244e791'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Нетопия',
 'salary': {'currency': 'RUR', 'max_salary': 250000.0, 'min_salary': 200000.0},
 'type_of_work': 'Полный день',
 'vacancy': 'Разработчик Python',
 'vacancy_link': 'https://hh.ru/vacancy/77091465',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f8d278121c6a068244e792'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Дубайт',
 'salary': {'currency': 'RUR', 'max_salary': 500000.0, 'min_salary': 300000.0},
 'type_of_work': 'Полный день',
 'vacancy': 'Разработчик C# / Python',
 'vacancy_link': 'https://hh.ru/vacancy/71147348',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f8d278121c6a068244e793'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Перфект Системс',
 'salary': {'currency': 'RUR', 'max_salary': 450000.0, 'min_salary': 300000.0},
 'type_of_work': 'Удаленная работа',
 'vacancy': 'Py

In [45]:
print_vacancy_by_salary_number(headhunter_collection, 100000, False)

Количество найденных вакансий с размером зарплаты < 100000: 639
Список найденных вакансий:

{'_id': ObjectId('63f8d278121c6a068244e795'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Завистовская Анжелика Сергеевна',
 'salary': {'currency': 'RUR', 'max_salary': 80000.0, 'min_salary': 0.0},
 'type_of_work': 'Полный день',
 'vacancy': 'Junior Python Backend Developer',
 'vacancy_link': 'https://hh.ru/vacancy/77372405',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f8d278121c6a068244e796'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Apostro',
 'salary': {'currency': 'USD', 'max_salary': 8000.0, 'min_salary': 0.0},
 'type_of_work': 'Удаленная работа',
 'vacancy': 'Ведущий Python разработчик (удаленка)',
 'vacancy_link': 'https://hh.ru/vacancy/76832091',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('63f8d278121c6a068244e798'),
 'city_of_work': 'Москва',
 'name_of_employer': 'Welltory',
 'salary': {'currency': '', 'max_salary': 0.0, 'min_salary': 0.0},
 'type_of_work': 'Удаленная